Import Libraries
TO DO
1) Describe Grid Location bottom left hand location and Cell Size
2) Plot the vector using x y coordinates.
3) Compare to Raster Stat Results, need to make sure that it only includes the cells fully inside (not on the border of the vector.

In [336]:
import numpy as np
print (np.__version__)
##from osgeo import gdal
from scipy import stats
import timeit
from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon
from osgeo import ogr
from math import ceil,floor

1.11.3


<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [337]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
shape = "data\\test.shp"
g = np.array(r.GetRasterBand(1).ReadAsArray())
csze = r.GetGeoTransform()[1]
ulX = r.GetGeoTransform()[0]
ulY = r.GetGeoTransform()[3]

Function to create data structure function takes in a grid with mxn and returns a grid of the same size summed by columns.  A vectorized version would be faster

In [339]:
def dtFZS(g,out):
    for xw in range(0,g.shape[1]):
        for yw in range(1,g.shape[0]):
            out[yw,xw] = out[yw-1,xw] + g[yw,xw]
    print("done")

In [340]:
##out = dtFZS(g)
DS_FstZS = np.zeros(g.shape,dtype='uint64')
dtFZS(g,DS_FstZS)


done


Fast Summary Statistics Helper Function

In [341]:
## Create a function that takes in x1, x2, ulX, and cllsze in world coords and returns list of x locations in grid coordinates
def x_world_to_grid(wx1,wx2,ulX,clSze):
    x1a = min(wx1,wx2)
    x2a = max(wx1,wx2)
    
    xstart = ceil  ((x1a - ulX - (clSze/2)) / clSze)
    xstop =  floor ((x2a - ulX - (clSze/2)) / clSze)

    return(list(range(xstart,xstop+1)))

## step1 = x_world_to_grid(1608450,1608490,ulX,30)
## print(step1)

In [343]:
## Create a Function that returns the centroid locations in world cordinates
## accepts a list as input

def x_grid_to_world(gx,ulX,clSze):
    return [i * clSze + ulX + (clSze/2) for i in gx]
    
## step2 = x_grid_to_world(step1,ulX,30)    
## print (step2)

In [345]:
## Create a function that takes in world coordinates of x_pred x1,x2,y1,y2 and returns y_pred as a list
def y_world_pred(wx1,wx2,wy1,wy2,wxpred):
    slope = 0;
    if (wx1 > wx2): ## switch Coords
        wx1,wx2,wy1,wy2 = wx2,wx1,wy2,wy1
    if(wx1 != wx2):    
        slope = (wy2-wy1)/(wx2-wx1)
    ##return [ i for i in xpred]
    return [(slope * (i-wx1)) + wy1 for i in wxpred]

## step3 = y_world_pred(1608450,1608490,2038038,2038070,step2)
## print(step3)

In [346]:
## Snap world y to grid coordinates wy is a list of cords
def y_world_to_grid(ulY,clSze,wypred):
        return [ceil((ulY - i - (clSze/2)) / clSze ) for i in wypred]

## step4 = y_world_to_grid(ulY,30,step3)
## print (step4)    

In [347]:
## Combine all the helper functions to return list of cells to sum

def xy_grid(wx1,wx2,ulX,wy1,wy2,ulY,clSze):
    ## Diagnosic Print
    ##print ("wx1 = " + str(wx1) + " wx2 = " + str(wx2) + " ulx = " +str(ulX))
    ##print ("wy1 = " + str(wy1) + " wy2 = " + str(wx2) + " uly = " +str(ulY))
    ##print("cellsize =" + str(clSze))
    xgrid  = x_world_to_grid(wx1,wx2,ulX,clSze)
    wxpred = x_grid_to_world(xgrid,ulX,clSze)  
    wypred = y_world_pred(wx1,wx2,wy1,wy2,wxpred)
    ygrid = y_world_to_grid(ulY,clSze,wypred)
    ##return xgrid
    return((ygrid,xgrid))


In [348]:
##  To DO
## This function needs to be rewritten to make v a lattice on the grid (not matching the grid locations but between the grid valeus)
## and to take coordinates

def FstZS(v,DS_FstZS,ulX,ulY,clSze):
    total = 0;    
    for p in range(1,(len(v)+0)):
        ## If moving left
        if v[p][0] > v[p-1][0]:
            cords = xy_grid(v[p][0],v[p-1][0],ulX,v[p][1],v[p-1][1],ulY,clSze)
            print(cords)
            total -= np.sum(DS_FstZS[cords])
        ## If moving Right
        elif v[p][0] < v[p-1][0]: 
            cords = xy_grid(v[p][0],v[p-1][0],ulX,v[p][1],v[p-1][1],ulY,clSze)
            print (cords)  
            total += np.sum(DS_FstZS[cords])
    return int(total)

In [349]:
v = [(1608590,2038038),(1608450,2038038),(1608450,2038138),(1608590,2038138),(1608590,2038038)]
#v = [(1608490,2038038),(1608450,2038038),(1608450,2038138),(1608490,2038138),(1608490,2038038)]


In [350]:
#g.shape
test = FstZS(v,DS_FstZS,ulX,ulY,csze)
test

([1509, 1509, 1509, 1509, 1509], [294, 295, 296, 297, 298])
([1506, 1506, 1506, 1506, 1506], [294, 295, 296, 297, 298])


1170

In [335]:
### Import Test vector
polygon = Polygon(v) 
zonal_stats(polygon,"data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif",stats= "sum")

C:\Users\smh362\AppData\Local\Continuum\Anaconda3\lib\site-packages\rasterstats\main.py:145: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
C:\Users\smh362\AppData\Local\Continuum\Anaconda3\lib\site-packages\rasterstats\io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)


[{'sum': 1170.0}]